In [1]:
# Collect data from Spotify
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy

# Data Visualization
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from matplotlib import colors
from termcolor import cprint
import plotly.express as px
import seaborn as sns
import warnings

# Data Preprocessing
from sklearn.preprocessing import StandardScaler
import pandas as pd

warnings.filterwarnings("ignore")

In [2]:
# setup api
cid = "5ce22bf9a89040eb895942284fe06912"
secret = "28251d136c45461e88942dd633250ae4"
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [5]:
# Step 1: Read the CSV file containing song data
df_songs = pd.read_csv('spotify_data_vn_all_ordered.csv')

# Step 2: Extract song IDs from the DataFrame
songs_list = []
for index, row in df_songs.iterrows():
    songs_data = {
        'id': row['id'],  # Adjust 'id' to match the column name in your CSV if different
    }
    songs_list.append(songs_data)

In [6]:
df_songs = pd.DataFrame(songs_list)

In [7]:
import time

audio_features = []

# Define the batch size
batch_size = 100

# Process IDs in batches
for i in range(0, len(df_songs['id']), batch_size):
    batch_ids = df_songs['id'][i:i + batch_size]
    try:
        results = sp.audio_features(batch_ids)
        for result in results:
            if result:  # Check if the result is not None
                audio_data = {
                    'id': result['id'],
                    'danceability': result['danceability'],
                    'energy': result['energy'],
                    'key': result["key"],
                    'loudness': result["loudness"],
                    'mode': result['mode'],
                    'speechiness': result['speechiness'],
                    'acousticness': result['acousticness'],
                    'instrumentalness': result['instrumentalness'],
                    'liveness': result['liveness'],
                    'valence': result['valence'],
                    'tempo': result['tempo'],
                    'time_signature': result['time_signature'],
                }
                audio_features.append(audio_data)
            else:
                print('Cannot retrieve audio features for song ID:', result['id'])
    except Exception as e:
        print(f"An error occurred: {e}")
        print('Cannot retrieve audio features for batch:', batch_ids)
        

    # Optional: Add a sleep interval to avoid hitting rate limits (429)
    time.sleep(1)

# Convert to DataFrame if needed
df_audio_features = pd.DataFrame(audio_features)


In [8]:
df_audios  = pd.DataFrame(audio_features)

In [9]:
# Merge audios and songs by ID
df = pd.merge(df_songs, df_audios, on='id')
# Save data into a CSV file
df.to_csv('Top_songs_2018-Now.csv', index=False)

In [10]:
# Import songs dataset
df_songs = pd.read_csv('Top_songs_2018-Now.csv')

In [11]:
df_songs.isnull().sum()

id                  0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
time_signature      0
dtype: int64